In [2]:
from functions import *
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", color_codes=True)

import pickle

In [5]:
database_name = 'bbc'

data = load_data(database_name)
data2 = SimplePreprocessing().transform(data.data)

[INFO][2023-06-06 23:28:55 - Mod: functions - Func: load_data - Line: 600]: Loaded data for bbc. 5 classes: ['sport', 'entertainment', 'tech', 'politics', 'business']. Number of documents: 2225.


In [8]:
import re 

num_w = [len(re.findall(r'\w+', sentence)) for sentence in data2]

In [11]:
sum(num_w)/(len(data2))

197.07280898876405

In [22]:
def load_bert1(database_name):
    
    try: 
        database = load_data(database_name=database_name)
        
        x_train = list(compress(database.data, database.is_train==1))
        y_train = list(compress(database.target, database.is_train==1))
        x_test = list(compress(database.data, database.is_train==0))
        y_test = list(compress(database.target, database.is_train==0))

        (x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(
            x_train,
            y_train,
            x_test, 
            y_test,
            class_names=database.target_names,
            preprocess_mode='bert',
            maxlen=350, 
            max_features=35000)
        
        database_name = 'classic4'
        path = f"./pickle_objects/models/model_bert_{database_name}.pickle"
        if os.path.isfile(path):            
            with open(path, 'rb') as f:
                learner = pickle.load(f)  
            logger.info(f'Load pre trained BERT model for dataset {database_name}.')    
            
        else:  
            print('oi')
#             bert_model = text.text_classifier("bert", train_data=(x_train, y_train),
#                                               preproc=preproc)
#             learner = ktrain.get_learner(bert_model, train_data=(x_train, y_train), 
#                                             batch_size=6)
#             learner.fit_onecycle(2e-5, 4)

#             with open(f"./pickle_objects/models/model_bert_{database_name}.pickle", "wb") as f:
#                 pickle.dump(learner, f, pickle.HIGHEST_PROTOCOL)
        
            #logger.info(f'Fitted BERT model on dataset {database_name}.')
        
        conf_table = learner.validate(val_data=(x_test, y_test),
                            class_names=database.target_names)
        
        return conf_table
  
    except Exception as e:
        logger.info(f'Error fitting BERT for {database_name}: \n {e}')



In [ ]:
with open(f'./pickle_objects/preprocess/tpbg/20ng/heterodata_pbg_20ng_tpbg_docf_None_train.pickle', 'rb') as f:
    heterodata_train = pickle.load(f)
with open(f'./pickle_objects/preprocess/tpbg/20ng/heterodata_pbg_20ng_tpbg_docf_None_test.pickle', 'rb') as f:
    heterodata_test = pickle.load(f)

In [ ]:
model = HeteroGNN(metadata=heterodata_train.metadata(), 
                  hidden_channels=100, 
                  out_channels=heterodata_train['source']['num_classes'],
                  num_layers=2,
                  p_dropout=0,
                  aggr='sum', 
                  version=2)
model

In [ ]:
heterodata_train

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


model.train()
optimizer.zero_grad()
out = model(heterodata_train.x_dict, heterodata_train.edge_index_dict)
prediction = out.argmax(1).cpu().numpy()

In [ ]:


database_name='webkb'
K=100
disable_tqdm=True

try:
    database_train = load_data(database_name=database_name, subset='train')
    database_test = load_data(database_name=database_name, subset='test')

    data_preprocessed_train = SimplePreprocessing().transform(database_train.data)
    data_preprocessed_test = SimplePreprocessing().transform(database_test.data)
    vectorizer = TfidfVectorizer()
    data_vectorized_train_fit = vectorizer.fit_transform(data_preprocessed_train)
    data_vectorized_test = vectorizer.transform(data_preprocessed_test)

    pbg_train = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
       local_threshold=1e-6, global_threshold=1e-6,
       feature_names=vectorizer.get_feature_names_out(), disable_tqdm=disable_tqdm)
    pbg_train.fit(data_vectorized_train_fit, database_train.target)

    predicted_target = pbg_train.predict(data_vectorized_test)
    micro_train = f1_score(predicted_target, database_test.target, average='micro')
    logger.info(f'Performance on pretrained pbg on {database_name} with K={K}. F1: {micro_train:.4f}.')

    vectorizer = TfidfVectorizer()
    data_vectorized_test_fit = vectorizer.fit_transform(data_preprocessed_test)
    data_vectorized_test = vectorizer.transform(data_preprocessed_test)
    pbg_test = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
           local_threshold=1e-6, global_threshold=1e-6,
           feature_names=vectorizer.get_feature_names_out(), disable_tqdm=disable_tqdm)
    pbg_test.fit(data_vectorized_test_fit, predicted_target)
    y_pred = pbg_test.predict(data_vectorized_test)
    micro_test = f1_score(y_pred, database_test.target, average='micro')

    logger.info(f'Loaded pbg for {database_name} with K={K}. F1: {micro_test:.4f}.')
    #return pbg_train, pbg_test

except Exception as e:
    logger.info(f'Error fitting pbg for {database_name}: \n {e}')    

#alpha=0.05, beta=0.0001, local_max_itr=5,
#global_max_itr=5

In [ ]:
pbg_train = UPBG(K, alpha=0.0005, beta=0.001, local_max_itr=50, global_max_itr=10,
   local_threshold=1e-6, global_threshold=1e-6,
   feature_names=vectorizer.get_feature_names_out(), disable_tqdm=False)
pbg_train.fit(data_vectorized_train_fit, database_train.target)

predicted_target = pbg_train.predict(data_vectorized_test)
micro_train = f1_score(predicted_target, database_test.target, average='micro')
logger.info(f'Performance on pretrained pbg on {database_name} with K={K}. F1: {micro_train:.4f}.')

vectorizer = TfidfVectorizer()
data_vectorized_test_fit = vectorizer.fit_transform(data_preprocessed_test)
data_vectorized_test = vectorizer.transform(data_preprocessed_test)
pbg_test = UPBG(K, alpha=0.0005, beta=0.001, local_max_itr=50, global_max_itr=10,
       local_threshold=1e-6, global_threshold=1e-6,
       feature_names=vectorizer.get_feature_names_out(), disable_tqdm=False)
pbg_test.fit(data_vectorized_test_fit, predicted_target)
y_pred = pbg_test.predict(data_vectorized_test)
micro_test = f1_score(y_pred, database_test.target, average='micro')

logger.info(f'Loaded pbg for {database_name} with K={K}. F1: {micro_test:.4f}.')


In [ ]:
pbg_test = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
       local_threshold=1e-6, global_threshold=1e-6,
       feature_names=vectorizer.get_feature_names_out(), disable_tqdm=disable_tqdm)


In [ ]:
pbg_test.fit(data_vectorized_test_fit, predicted_target)



In [ ]:
data_vectorized_test

In [ ]:
load_pbg_test(database_name='nsf', pbg_model_trained=pbg_train, K=200, disable_tqdm=True)

In [ ]:
y_pred = pbg_test.predict(data_vectorized_test)


In [ ]:
micro_test = f1_score(y_pred, database_test.target, average='micro')

logger.info(f'Loaded pbg for {database_name} with K={K}. F1: {micro_test:.4f}.')

In [ ]:
# number of documents NFS, Ohscal, WebKB, Industry Sector e MultiDomainSent

In [ ]:
# number of terms (WebKB, Reviews, Industry Sector, Review Polarity e Hitech)

In [ ]:
from nltk.corpus import webtext

In [ ]:
database_name = '20newsgroups'

pipe_lda = get_lda_train(database_name=database_name, K=100)
micro_lda = test_pipe(pipe_lda, database_name=database_name)
print(f'{database_name} - micro_lda: {micro_lda}')


pipe_nmf = get_nmf_train(database_name=database_name, K=100)
micro_nmf = test_pipe(pipe_nmf, database_name=database_name)
print(f'{database_name} - micro_nmf: {micro_nmf}')

#nsf - micro_lda: 0.9498721227621484
#nsf - micro_nmf: 0.9989769820971867

#classic4 - micro_lda: 0.9996476391825229
#classic4 - micro_nmf: 0.9992952783650458

#webkb - micro_lda: 0.9975852701479022
#webkb - micro_nmf: 0.9981889526109267

In [ ]:

### CARREGANDO DADOS  #######################

data_train = load_data(database_name='reuters', subset='train')
data_test = load_data(database_name='reuters', subset='test')

In [ ]:
data_train = load_data(database_name='nsf', subset='train')
data_test = load_data(database_name='nsf', subset='test')

print('preprocessing...')
pp = SimplePreprocessing()
M_train = pp.transform(data_train.data)
M_test = pp.transform(data_test.data)

print('done.')
vectorizer = TfidfVectorizer() #ngram_range=(1, 3)
M_train = vectorizer.fit_transform(M_train)
M_test = vectorizer.transform(M_test)

### PBG
K=100
pbg = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
           local_threshold=1e-6, global_threshold=1e-6,
           feature_names=vectorizer.get_feature_names_out())

#pbg = MultinomialNB(alpha=0.1)
#pbg = SVC()

print("fitting...")
pbg.fit(M_train, data_train.target)
print('done')


y_pred = pbg.predict(M_test)

score = f1_score(y_pred, data_test.target, average='micro')
print(f'f1 score: {score}')

        


In [ ]:
def run_pbg(database_name, K=100, disable_tqdm=True):
    
    try:
        database_train = load_data(database_name=database_name, subset='train')
        database_test = load_data(database_name=database_name, subset='test')

        data_preprocessed_train = SimplePreprocessing().transform(database_train.data)
        data_preprocessed_test = SimplePreprocessing().transform(database_test.data)
        vectorizer = TfidfVectorizer()
        data_vectorized_train_fit = vectorizer.fit_transform(data_preprocessed_train)
        data_vectorized_test = vectorizer.transform(data_preprocessed_test)
        
        pbg_train = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
           local_threshold=1e-6, global_threshold=1e-6,
           feature_names=vectorizer.get_feature_names_out(), disable_tqdm=disable_tqdm)
        pbg_train.fit(data_vectorized_train_fit, database_train.target)
        
        predicted_target = pbg_train.predict(data_vectorized_test)
        micro_train = f1_score(predicted_target, database_test.target, average='micro')
        logger.info(f'Performance on pretrained pbg on {database_name} with K={K}. F1: {micro_train:.4f}.')
        
        vectorizer = TfidfVectorizer()
        data_vectorized_test_fit = vectorizer.fit_transform(data_preprocessed_test)
        data_vectorized_test = vectorizer.transform(data_preprocessed_test)
        pbg_test = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
               local_threshold=1e-6, global_threshold=1e-6,
               feature_names=vectorizer.get_feature_names_out(), disable_tqdm=disable_tqdm)
        pbg_test.fit(data_vectorized_test_fit, predicted_target)
        y_pred = pbg_test.predict(data_vectorized_test)
        micro_test = f1_score(y_pred, database_test.target, average='micro')

        logger.info(f'Loaded pbg for {database_name} with K={K}. F1: {micro_test:.4f}.')
        return pbg_train, pbg_test
    
    except Exception as e:
        logger.info(f'Error fitting pbg for {database_name}: \n {e}')

# pbg_train, pbg_test = run_pbg(database_name='nsf', K=100, disable_tqdm=True)
# heterodata_pbg_nsf_k100_train = get_heterograph_pbg(pbg_train)
# heterodata_pbg_nsf_k100_test_full = get_heterograph_pbg(pbg_test)

In [ ]:
pbg_train, pbg_test = run_pbg(database_name='20newsgroups', K=100, disable_tqdm=True)
heterodata_pbg_nsf_k200_train = get_heterograph_pbg(pbg_train)
heterodata_pbg_nsf_k200_test_full = get_heterograph_pbg(pbg_test)

heterodata_pbg_nsf_k200_val, heterodata_pbg_nsf_k200_test, heterodata_pbg_nsf_k200_score = split_heterodata(heterodata_pbg_nsf_k200_test_full)


hidden_channels_list=[10, 50, 100, 200]
num_layers_list=[2, 3, 4]
p_dropout_list=[0.2]
patience=10

logger.info("Running experiments on nsf K=200")
df_experiment_nsf_k100 = experiment_gnn(database_name='nsf K=200',
                        heterodata_pbg_train=heterodata_pbg_nsf_k200_train,
                        heterodata_pbg_val=heterodata_pbg_nsf_k200_val,
                        heterodata_pbg_test=heterodata_pbg_nsf_k200_test,
                        hidden_channels_list=hidden_channels_list,
                        num_layers_list=num_layers_list,
                        p_dropout_list=p_dropout_list,
                        patience=patience,
                        verbose=False)

In [ ]:
x = experiment_gnn(database_name='nsf K=200',
                        heterodata_pbg_train=heterodata_pbg_nsf_k200_train,
                        heterodata_pbg_val=heterodata_pbg_nsf_k200_val,
                        heterodata_pbg_test=heterodata_pbg_nsf_k200_test,
                        hidden_channels_list=hidden_channels_list,
                        num_layers_list=num_layers_list,
                        p_dropout_list=p_dropout_list,
                        patience=100,
                        verbose=False)

x

In [ ]:
x

In [ ]:
x

In [ ]:
df_experiment_nsf_k100

In [ ]:
logger.info("Running experiments on nsf K=200")
df_experiment_nsf_k200 = experiment_gnn(database_name='nsf K=200',
                        heterodata_pbg_train=heterodata_pbg_nsf_k200_train,
                        heterodata_pbg_val=heterodata_pbg_nsf_k200_val,
                        heterodata_pbg_test=heterodata_pbg_nsf_k200_test,
                        hidden_channels_list=hidden_channels_list,
                        num_layers_list=num_layers_list,
                        p_dropout_list=p_dropout_list,
                        patience=patience,
                        verbose=False)

In [ ]:
heterodata_pbg_nsf_k100_val, heterodata_pbg_nsf_k100_test, heterodata_pbg_nsf_k100_score = split_heterodata(heterodata_pbg_nsf_k100_test_full)


In [ ]:
hidden_channels_list=[10, 50, 100, 200]
num_layers_list=[2, 3, 4]
p_dropout_list=[0.2]
patience=100

logger.info("Running experiments on nsf K=100")
df_experiment_nsf_k100 = experiment_gnn(database_name='nsf K=100',
                        heterodata_pbg_train=heterodata_pbg_nsf_k100_train,
                        heterodata_pbg_val=heterodata_pbg_nsf_k100_val,
                        heterodata_pbg_test=heterodata_pbg_nsf_k100_test,
                        hidden_channels_list=hidden_channels_list,
                        num_layers_list=num_layers_list,
                        p_dropout_list=p_dropout_list,
                        patience=patience,
                        verbose=False)

In [ ]:
df_experiment_nsf_k100

In [ ]:
pbg = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
           local_threshold=1e-6, global_threshold=1e-6,
           feature_names=vectorizer.get_feature_names_out())

#pbg = MultinomialNB(alpha=0.1)
#pbg = SVC()

print("fitting...")
pbg.fit(M_train, data_train.target)
print('done')


y_pred = pbg.predict(M_test)

score = f1_score(y_pred, data_test.target, average='micro')
print(f'f1 score: {score}')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from util import SimplePreprocessing
from upbg import UPBG
import numpy as np



data_train = load_data(database_name='reuters', subset='train')
data_test = load_data(database_name='reuters', subset='test')

print('preprocessing...')
pp = SimplePreprocessing()
M_train = pp.transform(data_train.data)
M_test = pp.transform(data_test.data)

print('done.')
vectorizer = TfidfVectorizer() #ngram_range=(1, 3)
M_train = vectorizer.fit_transform(M_train)
M_test = vectorizer.transform(M_test)

d = {'alt': 'alternative', 'comp': 'computer', 'os': 'operation system', 'ms-windows': 'windows', 'sys': 'system', 'x': 'interface',
     'misc': 'miscellaneous', 'rec': 'recreation', 'autos': 'automobile', 'sci': 'science', 'crypt': 'cryptography', 'med': 'medicine', 'soc': 'society'}
cls_names_ext = [[d.get(s, s) for s in name.split('.')]
                 for name in newsgroups_train.target_names]     

print(cls_names_ext)


### PBG
n_class = len(set(newsgroups_train.target))
print(f'nclass {n_class}')
K=100
pbg = UPBG(K, alpha=0.005, beta=0.001, local_max_itr=50, global_max_itr=10,
           local_threshold=1e-6, global_threshold=1e-6,
           feature_names=vectorizer.get_feature_names())

#pbg = MultinomialNB(alpha=0.1)
#pbg = SVC()

print("fitting...")
pbg.fit(M_train, newsgroups_train.target)
print('done')

pbg.print_top_topics(n_top_words=5, target_name=newsgroups_train.target_names)

y_pred = pbg.predict(M_test)

score = f1_score(y_pred, newsgroups_test.target, average='micro')
print(score)


In [ ]:
#BERTopic

from sklearn.datasets import fetch_20newsgroups
data = fetch_20newsgroups(subset='all')['data']

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(data, show_progress_bar=True)


from umap.umap_ import UMAP
umap_embeddings = UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)


import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)


import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

In [ ]:
docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

In [ ]:
news20 = fetch_20newsgroups()

X_train, X_test, y_train, y_test = train_test_split(
    news20.data[:5000], news20.target[:5000],
    stratify=news20.target[:5000])

X_train

In [ ]:
database = load_data(database_name="20newsgroups", subset="train")

database.data

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

def main():
    news20 = fetch_20newsgroups()

    X_train, X_test, y_train, y_test = train_test_split(
        news20.data[:5000], news20.target[:5000],
        stratify=news20.target[:5000])
    CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
    cv = CV(min_df=0.04, stop_words="english")
    lda = LDA(n_components=100, max_iter=30, n_jobs=-1)
    rfc = RFC(n_estimators=500, n_jobs=-1)
    estimators = [("cv", cv), ("lda", lda), ("rfc", rfc)]
    pl = Pipeline(estimators)

    pl.fit(X_train, y_train)
    y_pred = pl.predict(X_test)
    print(classification_report(
        y_test, y_pred, target_names=news20.target_names))

if __name__ == "__main__":
    main()


In [ ]:
!pip uninstall umap -Y

In [ ]:
import ktrain
from ktrain import text

In [ ]:
def load_bert(database_name):
    
    try: 
        database_train = load_data(database_name=database_name, subset="train")
        database_test = load_data(database_name=database_name, subset="test")
        
        (x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(
            x_train=database_train.data,
            y_train=database_train.target,
            x_test=database_test.data, 
            y_test=database_test.target,
            class_names=database.target_names,
            preprocess_mode='bert',
            maxlen=350, 
            max_features=35000)
        
        bert_model = text.text_classifier("bert", train_data=(x_train, y_train),
                                          preproc=preproc)
        learner = ktrain.get_learner(bert_model, train_data=(x_train, y_train), 
                                        batch_size=6)
        learner.fit_onecycle(2e-5, 4)
        
        with open(f"./pickle_objects/model_bert_{database_name}.pickle", "wb") as f:
            pickle.dump(learner, f, pickle.HIGHEST_PROTOCOL)
        
        logger.info(f'Fitted BERT model on dataset {database_name}.')
        conf_table = learner.validate(val_data=(x_test, y_test),
                            class_names=database_train.target_names)
        
        return conf_table
  
    except Exception as e:
        logger.info(f'Error fitting pbg for {database_name}: \n {e}')

In [ ]:
load_bert(database_name='reuters')
load_bert(database_name='bbc_news')

In [ ]:
database_train = load_data(database_name="20newsgroups", subset="train")
database_test = load_data(database_name="20newsgroups", subset="test")

(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(
    x_train=database_train.data,
    y_train=database_train.target,
    x_test=database_train.data, 
    y_test=database_train.target,
    class_names=database_train.target_names,
    preprocess_mode='bert',
    maxlen=350, 
    max_features=35000)

bert_model = text.text_classifier("bert", train_data=(x_train, y_train),
                                  preproc=preproc)
learner = ktrain.get_learner(bert_model, train_data=(x_train, y_train), 
                                batch_size=6)
learner.fit_onecycle(2e-5, 4)

logger.info(f'Fitted BERT model on dataset {database_name}.')
x = learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

In [ ]:
x = learner.validate(val_data=(x_test, y_test))

In [ ]:
x

In [ ]:
https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6
    
    
    https://maartengr.github.io/BERTopic/getting_started/topicsperclass/topicsperclass.html